In [2]:
import keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
x_train = []
x_test = []
y_train = []
y_test = []
#df_test = pd.read_csv('target_to_number.txt')
label_map = {   'PASS': 0,
                'NG': 1}

dim = 256

In [4]:
# Preparing training data
dirs = os.listdir("../../../bt081-group12/FINIAL_train_data/")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("../../../bt081-group12/FINIAL_train_data/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('../../../bt081-group12/FINIAL_train_data/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(15)
        targets = [label_map[dirs[k]]]
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)   
y_train = np.array(y_train)
x_train = np.array(x_train)

100%|██████████| 2/2 [00:03<00:00,  1.32s/it]


In [5]:
print(x_train.shape)
print(y_train.shape)


(624, 256, 256, 3)
(624, 1)


In [6]:
batch_size = 20
num_classes = 2
epochs = 100
save_dir = os.path.join(os.getcwd(), 'saved_models') #獲取當前目錄，並組合成新目錄
model_name = 'R4_200_400_DenseNet201_RT_model.h5'

In [7]:
save_dir, model_name

('/project/bt081-group12/yolo/keras-yolo3/saved_models',
 'R4_200_400_DenseNet201_RT_model.h5')

In [8]:
# x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) # Want a balanced split for all the classes
for train_index, test_index in sss.split(x_train, y_train):
    print("Using {} for training and {} for testing".format(len(train_index), len(test_index)))
    x_train, x_test = x_train[train_index], x_train[test_index]
    y_train, y_test = y_train[train_index], y_train[test_index]
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Using 499 for training and 125 for testing
x_train shape: (499, 256, 256, 3)
x_test shape: (125, 256, 256, 3)
y_train shape: (499, 1)
y_test shape: (125, 1)
499 train samples
125 test samples


In [9]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#print(y_train)
#print(y_test)

In [10]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    rotation_range=35,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [11]:
# Use ModelCheckpoint to save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)

# earlystop
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

learning_rate = 0.0005

callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=0), 
              ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

    


/opt/conda/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [12]:
base_model = DenseNet201(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [13]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

In [ ]:
model_history=model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=datagen.flow(x_test, y_test, batch_size=batch_size), 
                    validation_steps=len(x_test)/batch_size,
                    callbacks=callbacks,
                    epochs=epochs, 
                    verbose=1)

Epoch 1/100


In [17]:
model_name2 = 'R4_200_400__Xception_RT_model.h5'

In [18]:
# Use ModelCheckpoint to save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name2)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)

# earlystop
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

learning_rate = 0.0005

callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=0), 
              ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]


# Freeze layers not in classifier due to loading imagenet weights
# for layer in base_model.layers:
#     layer.trainable = False



# Load any existing weights
# if os.path.isfile(weights):
#     model.load_weights(weights)

/opt/conda/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [19]:
base_model2 = Xception(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg') # Average pooling reduces output dimensions
x = base_model2.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model2.input, outputs=predictions)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [22]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

In [23]:
model_history2=model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=datagen.flow(x_test, y_test, batch_size=batch_size), 
                    validation_steps=len(x_test)/batch_size,
                    callbacks=callbacks,
                    epochs=epochs, 
                    verbose=1)

Epoch 1/100
25/24 [==============================] - 24s 979ms/step - loss: 0.2860 - acc: 0.8739 - val_loss: 0.9732 - val_acc: 0.7680
Epoch 2/100
25/24 [==============================] - 8s 335ms/step - loss: 0.1175 - acc: 0.9660 - val_loss: 0.0862 - val_acc: 0.9680
Epoch 3/100
25/24 [==============================] - 8s 332ms/step - loss: 0.0833 - acc: 0.9799 - val_loss: 0.4526 - val_acc: 0.8880
Epoch 4/100
25/24 [==============================] - 8s 334ms/step - loss: 0.0737 - acc: 0.9800 - val_loss: 0.2116 - val_acc: 0.9840
Epoch 5/100
25/24 [==============================] - 8s 337ms/step - loss: 0.0196 - acc: 0.9920 - val_loss: 0.2078 - val_acc: 0.9840
Epoch 6/100
25/24 [==============================] - 8s 332ms/step - loss: 0.0250 - acc: 0.9920 - val_loss: 0.1101 - val_acc: 0.9840
Epoch 7/100
25/24 [==============================] - 8s 334ms/step - loss: 0.0219 - acc: 0.9940 - val_loss: 0.1428 - val_acc: 0.9840
Epoch 8/100
25/24 [==============================] - 8s 336ms/step -

In [46]:
model_name3 = 'R4_200_400__ResNet50_RT_model.h5'

In [47]:
# Use ModelCheckpoint to save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name3)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)

# earlystop
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

learning_rate = 0.0005

callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=0), 
              ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

/opt/conda/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [48]:
base_model3 = Xception(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg') # Average pooling reduces output dimensions
x = base_model3.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model3.input, outputs=predictions)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [22]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

In [49]:
y_train.argmax(axis=1).sum()

179

In [24]:
model_history3=model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=datagen.flow(x_test, y_test, batch_size=batch_size), 
                    validation_steps=len(x_test)/batch_size,
                    callbacks=callbacks,
                    epochs=epochs, 
                    verbose=1)

Epoch 1/100
25/24 [==============================] - 20s 806ms/step - loss: 0.2978 - acc: 0.8755 - val_loss: 0.4069 - val_acc: 0.9200
Epoch 2/100
25/24 [==============================] - 8s 335ms/step - loss: 0.1435 - acc: 0.9540 - val_loss: 0.2698 - val_acc: 0.9600
Epoch 3/100
25/24 [==============================] - 8s 334ms/step - loss: 0.1238 - acc: 0.9620 - val_loss: 0.1018 - val_acc: 0.9760
Epoch 4/100
25/24 [==============================] - 8s 338ms/step - loss: 0.0670 - acc: 0.9820 - val_loss: 0.1805 - val_acc: 0.9520
Epoch 5/100
25/24 [==============================] - 8s 337ms/step - loss: 0.0555 - acc: 0.9820 - val_loss: 0.2213 - val_acc: 0.9600
Epoch 6/100
25/24 [==============================] - 8s 337ms/step - loss: 0.0279 - acc: 0.9900 - val_loss: 0.0481 - val_acc: 0.9840
Epoch 7/100
25/24 [==============================] - 8s 338ms/step - loss: 0.0134 - acc: 0.9959 - val_loss: 0.0233 - val_acc: 0.9920
Epoch 8/100
25/24 [==============================] - 8s 339ms/step -

In [13]:

# loading our save model
print("Loading trained model")
model = load_model('../../../bt081-group12/model/DenseNet201_RT_model.h5')

#model = model.save('whereami_model.h5')
#model = load_model('whereami_model.h5')



Loading trained model


In [14]:
model_1 = load_model('../../../bt081-group12/model/ResNet50_RT_model.h5')

In [15]:
model_2 = load_model('../../../bt081-group12/model/Xception_RT_model.h5')

In [16]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
scores1 = model_1.evaluate(x_test, y_test, verbose=1)
scores2 = model_2.evaluate(x_test, y_test, verbose=1)
final_pred = np.array([])

125/125 [==============================] - 5s 41ms/step


In [17]:
#from scipy.stats import mode
#print(len(x_test))
#final_pred
#for i in range(0,len(x_test)):
#    final_pred =np.append(final_pred, mode([scores[i], scores1[i], scores2[i]]))

In [18]:
x

<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 256) dtype=float32>

training_loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.plot(training_loss, label="training_loss")
plt.plot(val_loss, label="validation_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

training_acc = model_history.history['acc']
val_acc = model_history.history['val_acc']

plt.plot(training_acc, label="training_acc")
plt.plot(val_acc, label="validation_acc")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [19]:
final_pred = np.array([])
for folder_path, folder_list, file_list in os.walk('../../../bt081-group12/validation_100_ImageEnhance/'):
    print(folder_list)
    print(folder_path)
    
    if not folder_list:
        x = np.asarray([cv2.resize(cv2.imread(os.path.join(folder_path, filename), cv2.IMREAD_COLOR), (256,256)) for filename in file_list])
        print(x.shape)
x_test = np.array(x, np.float32)

#print(x_test)
#x_test /= 255.
print(x_test.shape)
file_list= [ file_list[:-4] for file_list in file_list if file_list[-4:]=='.tif']

[]
../../../bt081-group12/validation_100_ImageEnhance/
(100, 256, 256, 3)
(100, 256, 256, 3)


In [20]:
p_test_DenseNet201 = model.predict(x_test, verbose=1)
p_test_ResNet50 = model_1.predict(x_test, verbose=1)
p_test_Xception = model_2.predict(x_test, verbose=1)
preds = []


100/100 [==============================] - 5s 51ms/step


In [21]:
for i in range(len(p_test_DenseNet201)):
#    print(p_test_DenseNet201[i,1])
    
    if p_test_DenseNet201[i,1]>0.5:
        pose=1
    else:
        pos = np.argmax(p_test_DenseNet201[i]) # = p_test_DenseNet201[i,1]>0.5
#    print(pos)
    preds.append(list(label_map.values())[list(label_map.values()).index(pos)])

#    pos = np.argmax(p_test_DenseNet201[i]) # = p_test_DenseNet201[i,1]>0.5
#    preds.append(list(label_map.values())[list(label_map.values()).index(pos)])

print(preds)
#df_test = pd.DataFrame({"class":preds,"id":file_list})
#df_test.to_csv('DenseNet201_R3_submission8.csv', index=False)    
    


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [22]:
preds2=[]
for i in range(len(p_test_ResNet50)):
    print(p_test_ResNet50[i,1])
    if p_test_ResNet50[i,1]>0.5:
        pos=1
    else:
        pos = np.argmax(p_test_ResNet50[i]) # = p_test_DenseNet201[i,1]>0.5
    print(pos)
    pos = np.argmax(p_test_ResNet50[i])
    preds2.append(list(label_map.values())[list(label_map.values()).index(pos)])
    
df_test = pd.DataFrame({"class":preds2,"id":file_list})
df_test.to_csv('ResNet50_R1_submission7.csv', index=False)   
    
#    pos = np.argmax(p_test_ResNet50[i]) # = p_test_DenseNet201[i,1]>0.5
#    preds2.append(list(label_map.values())[list(label_map.values()).index(pos)])

6.0688603e-07
0
4.0197752e-05
0
2.4392879e-05
0
0.021998584
0
4.6548885e-06
0
0.11090138
0
3.8153466e-05
0
0.00013839033
0
1.6272617e-05
0
0.0059876475
0
3.642444e-06
0
0.0034293432
0
0.0011261833
0
0.0027226382
0
0.0016369303
0
0.0069998465
0
6.2636177e-06
0
3.1957043e-06
0
0.79125285
1
0.5836178
1
3.3271892e-05
0
1.8173598e-05
0
3.667558e-07
0
1.9822664e-06
0
2.1375814e-05
0
4.0326802e-05
0
1.9322984e-08
0
6.636871e-07
0
4.383238e-07
0
8.1596234e-05
0
7.838206e-06
0
5.9310652e-05
0
0.008504449
0
0.7020528
1
8.230701e-05
0
4.649404e-07
0
0.00093627
0
0.00010964431
0
2.7486994e-05
0
1.8154866e-07
0
2.699421e-05
0
4.8443206e-05
0
0.27864158
0
0.00024215432
0
0.015373518
0
4.82141e-06
0
0.0005713679
0
1.8576125e-06
0
0.79125285
1
0.0011544385
0
0.011115565
0
0.009765608
0
2.5881977e-08
0
0.9930555
1
0.00012591541
0
0.016039163
0
4.0168793e-06
0
3.1545792e-07
0
2.1547508e-05
0
0.0015019878
0
2.2253512e-06
0
2.4647103e-05
0
0.009583507
0
0.07446475
0
3.9433482e-05
0
2.5261674e-05
0
5.80723

In [23]:
preds3=[]
for i in range(len(p_test_Xception)):
    print(p_test_Xception[i,1])
    if p_test_ResNet50[i,1]>0.5:
        pos=1
    else:
        pos = np.argmax(p_test_Xception[i]) # = p_test_DenseNet201[i,1]>0.5
    print(pos)
    preds3.append(list(label_map.values())[list(label_map.values()).index(pos)])
    
#    pos = np.argmax(p_test_Xception[i]) # = p_test_DenseNet201[i,1]>0.5
#    preds3.append(list(label_map.values())[list(label_map.values()).index(pos)])
df_test = pd.DataFrame({"class":preds3,"id":file_list})
df_test.to_csv('Xception_R1_submission7.csv', index=False)      

1.3828836e-05
0
0.00053954526
0
0.00018015214
0
0.014509088
0
1.9557342e-06
0
5.634636e-05
0
9.012474e-06
0
0.004206936
0
0.0013139129
0
0.00048930594
0
7.978863e-06
0
0.0112366695
0
0.00071553024
0
0.020093247
0
0.0004690105
0
5.6910303e-05
0
3.8330036e-06
0
3.3424767e-06
0
0.005858261
1
0.022359572
1
9.4481526e-05
0
0.0003687597
0
7.578715e-06
0
0.00048261898
0
8.033828e-05
0
1.965605e-08
0
8.605594e-05
0
9.69176e-06
0
8.468222e-05
0
0.00030093783
0
6.7737885e-05
0
0.00013488854
0
0.0057307463
0
0.33747056
1
1.1762501e-05
0
5.577259e-05
0
0.00024940437
0
0.0007241494
0
6.374349e-05
0
4.7355247e-06
0
0.00016848509
0
2.5837846e-05
0
0.0028740675
0
0.0025625618
0
0.0008728734
0
4.990994e-06
0
0.0014241793
0
0.00032644172
0
0.005858261
1
6.840983e-05
0
0.009845403
0
0.018501367
0
9.3276833e-07
0
0.76302546
1
0.00095160946
0
0.0061603184
0
1.9340887e-06
0
3.2437206e-06
0
1.7578035e-05
0
0.00038348933
0
2.864381e-05
0
4.286299e-05
0
0.051735103
0
0.11572043
0
1.1970325e-05
0
9.892266e-06
0

In [24]:
final_pred = np.array([])
final_pred
for i in range(0,len(x_test)):
    final_pred =np.append(final_pred, np.argmax(np.bincount([preds[i], preds2[i], preds3[i]])))

In [25]:
#from sklearn.metrics import confusion_matrix

#confusion_matrix()

In [34]:
df_test = pd.DataFrame({"class":final_pred,"id":file_list})
df_test.to_csv('R1_200_400_submission10.csv', index=False)

In [40]:
path='../../../bt081-group12/validation_100_ImageEnhance/'
f = os.listdir(path)

In [41]:
print(f)

['0059.tif', '0070.tif', '0036.tif', '0052.tif', '0001.tif', '0012.tif', '0047.tif', '0083.tif', '0086.tif', '0077.tif', '0081.tif', '0006.tif', '0017.tif', '0055.tif', '0042.tif', '0080.tif', '0093.tif', '0040.tif', '0050.tif', '0010.tif', '0048.tif', '0095.tif', '0028.tif', '0002.tif', '0054.tif', '0013.tif', '0061.tif', '0068.tif', '0094.tif', '0038.tif', '0008.tif', '0071.tif', '0056.tif', '0026.tif', '0033.tif', '0090.tif', '0066.tif', '0031.tif', '0018.tif', '0019.tif', '0053.tif', '0085.tif', '0082.tif', '0062.tif', '0027.tif', '0039.tif', '0078.tif', '0043.tif', '0029.tif', '0084.tif', '0069.tif', '0072.tif', '0032.tif', '0098.tif', '0045.tif', '0065.tif', '0092.tif', '0074.tif', '0023.tif', '0022.tif', '0100.tif', '0060.tif', '0003.tif', '0088.tif', '0015.tif', '0073.tif', '0076.tif', '0064.tif', '0041.tif', '0009.tif', '0025.tif', '0097.tif', '0058.tif', '0057.tif', '0091.tif', '0020.tif', '0034.tif', '0063.tif', '0030.tif', '0096.tif', '0087.tif', '0049.tif', '0046.tif', '00

In [43]:
yolov3_input_pred = []
yolov3_input_pred_number = []
#[x_test[i] for i in range(len(final_pred)) if final_pred[i]==1]
for i in range(len(final_pred)):
    if final_pred[i]==1:
        x_test[i]
        yolov3_input_pred.append(x_test[i])
        yolov3_input_pred_number.append(f[i])
#        print(x_test.shape)
yolov3_input_pred = np.array(yolov3_input_pred)  
yolov3_input_pred_number=np.array(yolov3_input_pred_number)

In [44]:
yolov3_input_pred.shape

(7, 256, 256, 3)

In [47]:
yolov3_input_pred_number

array(['0050.tif', '0010.tif', '0026.tif', '0029.tif', '0098.tif',
       '0009.tif', '0091.tif'], dtype='<U8')

In [48]:
yolov3_input_pred_number[3]

'0029.tif'

In [49]:
len(yolov3_input_pred)

7

In [50]:
import os
import argparse
import json
import cv2
from utils.utils import get_yolo_boxes, makedirs
from utils.bbox import draw_boxes
from keras.models import load_model
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
output_path  = 'output/'
makedirs(output_path)

# Set some parameter
net_h, net_w = 416, 416 # a multiple of 32, the smaller the faster
obj_thresh, nms_thresh = 0.5, 0.45
anchors = [55,69, 75,234, 133,240, 136,129, 142,363, 203,290, 228,184, 285,359, 341,260]
labels = ["NG"]

# Load the model
infer_model = load_model('NG.h5')
# Predict bounding boxes 

    # the main loop

for i in range(len(yolov3_input_pred)):
    image = yolov3_input_pred[i]
#    print(image)
# predict the bounding boxes
    boxes = get_yolo_boxes(infer_model, [image], net_h, net_w, anchors, obj_thresh, nms_thresh)[0]

# draw bounding boxes on the image using labels
    draw_boxes(image, boxes, labels, obj_thresh) 


# write the image with bounding boxes to file
    output_img_path = yolov3_input_pred_number[i]
    cv2.imwrite(output_img_path, np.uint8(image))
#    img = cv2.imread(output_img_path)[:,:,::-1]
#    plt.imshow(img)

/opt/conda/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
